In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import sys, os
import matplotlib.pyplot as plt

# 2018-12-13 Prepare batch downloads
So far, I understood that I need to process the data files in a way that will generate homogeneous files that we will then be able to process and feed to the training of the AI.

Here, I want to prepare the files and directories containing the processed and raw files.

We will create a directory structure that reflect the various experiments that we will parse and analyse.

In [ ]:
# directories and files
tissue_ai_rootdir = "../"
datadir = "%s/data"%(tissue_ai_rootdir)
md_fname = "%s/metadata.txt"%(datadir)
md = pd.read_csv(md_fname, sep='\t', low_memory=False)
fastqs = md.loc[md["File format"] == "fastq"]

Now `fastqs` is a variable that contains a Pandas Data.Frame with all the FASTQ files that we will download and process. As we saw in the previous notebook, there are single-ended and paired-ended files. Let's group the files together by experiment first.

In [ ]:
# the collection of all the experiment accession IDs
experiment_names = fastqs["Experiment accession"].unique()

In [ ]:
# iterate over all the experiments with an unique identifier
for experiment_name in experiment_names :
    
    # get the rows in the metadata table corresponding to this experiment
    experiment = fastqs.loc[fastqs["Experiment accession"] ==  experiment_name]
    
    # this list will allow us to keep track of which samples we have or have not
    # processed
    samples_processed = []
    
    # iterate over all the samples in the experiment
    for index, sample in experiment.iterrows() :
        
        # keep track of the sub-sample
        replicate_idx = 1
        
        # this is the case in which the sample is paired-ended: we need to find
        # which sample corresponds to the pair
        if sample["Run type"] == "paired-ended" :
            
            # the identifier of the sample allows us to know which sample is paired
            # to the one we're visiting
            id_read1 = sample["File accession"]
            sample_paired = experiment.loc[experiment["Paired with"] ==  id_read1].iloc[0]
            id_read2 = sample_paired["File accession"]
                        
            # if we haven't already processed the sample, print the results
            if id_read1 not in samples_processed :
                print "paired-ended"
                print id_read1, sample["File download URL"]
                print id_read2, sample_paired["File download URL"]

            # add the paired samples to the list of samples that we processed
            samples_processed.append(id_read1)
            samples_processed.append(id_read2)
        elif sample["Run type"] ==  "single-ended" :
            print "single-ended"
            print sample["File accession"], sample["File download URL"]
        
        # increment the replicate index
        replicate_idx += 1

These lines of code will do the trick. Now there is the question on how to quantify the expression from single-ended RNA-seq experiments. Kallisto wants the fragment size (average and standard deviation) as input to its algorithm. However, it might be difficult to extract that information from large amounts of experiments.

I read online that there is a better alternative: salmon.

I ran the quantification algorithm from salmon. I want to compare the results of the two algorithms.

I will switch to an R notebook to do this.